In [1]:
import geopandas as gpd
import rasterio
import os
from shapely.geometry import box

In [4]:
# Load your AOI shapefile
aoi = gpd.read_file("willow_plot_aoi.shp")

In [5]:
# Directory containing the TIFFs
tif_dir = r"C:\2024 Willow Release Height Project\YNP LiDAR\DEM_TIFFS"

# List to store filenames of TIFFs within the AOI
tifs_in_aoi = []


In [7]:


# Find the first TIFF file in the directory
for filename in os.listdir(tif_dir):
    if filename.endswith(".tif"):
        first_tif_path = os.path.join(tif_dir, filename)
        break

# Open the first TIFF and print its CRS
with rasterio.open(first_tif_path) as src:
    print("CRS of the first TIFF:", src.crs)


CRS of the first TIFF: COMPD_CS["NAD83(2011) / UTM zone 12N + NAVD88 height",PROJCS["NAD83(2011) / UTM zone 12N",GEOGCS["NAD83(2011)",DATUM["NAD83_National_Spatial_Reference_System_2011",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","1116"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","6318"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-111],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","6341"]],VERT_CS["NAVD88 height",VERT_DATUM["North American Vertical Datum 1988",2005,AUTHORITY["EPSG","5103"]],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Gravity-related height",UP],AUTHORITY["EPSG","5703"]]]


In [8]:
aoi.crs

<Compound CRS: COMPD_CS["NAD83(2011) / UTM zone 12N + NAVD_1988_G ...>
Name: NAD83(2011) / UTM zone 12N + NAVD_1988_Geoid18
Axis Info [cartesian|vertical]:
- [east]: Easting (metre)
- [north]: Northing (metre)
- [up]: Gravity-related height (metre)
Area of Use:
- undefined
Datum: NAD83 (National Spatial Reference System 2011)
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich
Sub CRS:
- NAD83(2011) / UTM zone 12N
- NAVD_1988_Geoid18

In [9]:
import time

start = time.time()

# Open the file for writing at the start of the process
with open("tifs_in_aoi_dem.txt", "w") as file:
    # Iterate over files in the directory
    for filename in os.listdir(tif_dir):
        if filename.endswith(".tif"):
            filepath = os.path.join(tif_dir, filename)
            with rasterio.open(filepath) as src:
                # Create a bounding box from the TIFF's bounds
                tif_bounds = box(*src.bounds)
                # Check if the bounding box intersects with the AOI
                if aoi.intersects(tif_bounds).any():
                    file.write(f"{filename}\n")

print("Process completed. The list of TIFFs in the AOI has been saved.")

end  = time.time()

elapsed_time = end - start

print(f"Elapsed time: {elapsed_time} seconds")

Process completed. The list of TIFFs in the AOI has been saved.
Elapsed time: 386.01562333106995 seconds


In [10]:
from rasterio.merge import merge

# Path to the text file containing the TIFF filenames in the AOI
filename_list_path = 'tifs_in_aoi_dem.txt'

# Base directory containing the TIFFs
base_tif_dir = r"C:\2024 Willow Release Height Project\YNP LiDAR\DEM_TIFFS"

# Read the list of filenames from the file
with open(filename_list_path, 'r') as file:
    tiff_files = [line.strip() for line in file]

# List to hold open raster files for merging
src_files_to_mosaic = []

# Open the TIFF files specified in the list
for filename in tiff_files:
    file_path = os.path.join(base_tif_dir, filename)
    src = rasterio.open(file_path)
    src_files_to_mosaic.append(src)

In [11]:
# Perform the merge operation
mosaic, out_trans = merge(src_files_to_mosaic)

# Output metadata
out_meta = src.meta.copy()

# Update the metadata to reflect the dimensions and transformation of the mosaic
out_meta.update({
    "driver": "GTiff",
    "height": mosaic.shape[1],
    "width": mosaic.shape[2],
    "transform": out_trans,
    "crs": src.crs
})

# Write the mosaic raster to disk
with rasterio.open(r'C:\2024 Willow Release Height Project\YNP LiDAR\Max_surface_height_rasters\merged_AOI\merged_DEM_AOI.tif', "w", **out_meta) as dest:
    dest.write(mosaic)

# Close the source files
for src in src_files_to_mosaic:
    src.close()

print("Merging complete. The merged file is saved.")

Merging complete. The merged file is saved.


In [12]:
from rasterio.enums import Resampling

# Paths to your CHM and DEM raster files
chm_raster_path = r'C:\2024 Willow Release Height Project\YNP LiDAR\Max_surface_height_rasters\merged_AOI\merged_CHM_AOI.tif'
dem_raster_path = r'C:\2024 Willow Release Height Project\YNP LiDAR\Max_surface_height_rasters\merged_AOI\merged_DEM_AOI.tif'
output_vhm_path = r'C:\2024 Willow Release Height Project\YNP LiDAR\Max_surface_height_rasters\merged_AOI\merged_VHM_AOI.tif'

# Open the CHM and DEM rasters
with rasterio.open(chm_raster_path) as src_chm, rasterio.open(dem_raster_path) as src_dem:
    # Read the rasters as arrays
    chm_array = src_chm.read(1)  # Read the first band
    dem_array = src_dem.read(1)  # Read the first band

    # Check if both rasters have the same shape
    if chm_array.shape != dem_array.shape:
        # Resample DEM to match CHM (if they differ in shape or transform)
        dem_array = src_dem.read(
            1,
            out_shape=(src_chm.height, src_chm.width),
            resampling=Resampling.bilinear
        )

    # Calculate the Vegetation Height Model by subtracting DEM from CHM
    vhm_array = chm_array - dem_array

    # Output metadata (using CHM's metadata)
    out_meta = src_chm.meta.copy()

    # Create the VHM raster file
    with rasterio.open(output_vhm_path, 'w', **out_meta) as dst:
        dst.write(vhm_array, 1)  # Write the VHM array to the first band

print("Vegetation Height Model created successfully.")

Vegetation Height Model created successfully.
